In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline 

In [3]:
from yaml import load as yaml_load
import os
import sys
import json
sys.path.append('../')
import findspark
findspark.init()
import pyspark
from pyspark.sql.types import LongType
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StandardScaler, VectorAssembler, VectorIndexer, StandardScaler
from pyspark.sql.functions import *
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, udf
from pyspark.sql.types import LongType, StringType, FloatType, IntegerType, DoubleType
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
import re
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml import Pipeline
spark = SparkSession.builder.master("local[*]").appName("Data cleaning").getOrCreate()

In [4]:
from src.data.make_dataset import LoadDataframe
from src.features.build_features import extract_features_classification

In [5]:
def _load_config_file(config_file):
    """
    Load configuration file
    :param config_file: is the configuration file
    :return: configuration
    :rtype: dict
    """
    with open(config_file) as yml_config:
        return yaml_load(yml_config)

def _build_configuration(config_file):
    """
    Build the operation configuration dict
    :param config_file: is the path to the yaml config_file
    :type: string
    :return: config: global configuration
    :rtype dict
    """
    # yaml config
    config = _load_config_file(config_file)
    return config
def visualisation_prediction(y_test, y_pred):
    import matplotlib
    import matplotlib.pyplot as plt
    matplotlib.rc('xtick', labelsize=30) 
    matplotlib.rc('ytick', labelsize=30) 
    fig, ax = plt.subplots(figsize=(50, 40))
    plt.style.use('ggplot')
    plt.plot(y_pred, y_test, 'ro')
    plt.xlabel('Predicted Crime', fontsize = 30)
    plt.ylabel('Actual Crime', fontsize = 30)
    plt.title('Predicted Y (Crimes) to the Actual Y (Crimes)', fontsize = 30)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
    

config_file = "/home/ml/Documents/crimes_chigaco/config/config.yml"
config = _build_configuration(config_file)

In [6]:
%%time
obj_df_loaded = LoadDataframe(config, '2013', '2014')


CPU times: user 290 µs, sys: 27 µs, total: 317 µs
Wall time: 322 µs


In [7]:
df_temp = obj_df_loaded.df_temperature()
df_sky  = obj_df_loaded.df_sky()
df_socio = obj_df_loaded.df_socio()
df_crimes_socio = obj_df_loaded.df_crime_socio()
obj_extract_features_classification = extract_features_classification(config, df_crimes_socio, df_temp, df_sky)

/usr/lib/spark/spark-2.4.0-bin-hadoop2.7/python/pyspark/sql/dataframe.py:1793: UserWarning: to_replace is a dict and value is not None. value will be ignored.
  warnings.warn("to_replace is a dict and value is not None. value will be ignored.")


In [8]:
from pyspark.sql.functions import isnan, when, count, col, isnull

In [9]:
df_ml = obj_extract_features_classification.extract_feature()

In [10]:
df_ml.limit(5).toPandas().T

,0,1,2,3,4
year,2013,2013,2013,2013,2013
month,11,12,6,2,2
day,7,2,10,7,19
hour,8,4,10,15,15
community_area_number,43,58,44,54,54
id,10754746,9412469,9166760,9001193,9015161
cas_number,HZ517281,HW555076,HW312239,HW147978,HW162837
date,2013-11-07 08:00:00,2013-12-02 04:28:00,2013-06-10 10:20:00,2013-02-07 15:45:00,2013-02-19 15:45:00
block,019XX E 75TH ST,028XX W 48TH PL,080XX S COTTAGE GROVE AVE,130XX S LANGLEY AVE,130XX S LANGLEY AVE
iucr,0890,0560,0486,0460,0460


In [11]:
df_ml.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- community_area_number: string (nullable = true)
 |-- id: string (nullable = true)
 |-- cas_number: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- block: string (nullable = true)
 |-- iucr: string (nullable = true)
 |-- primary_type: string (nullable = true)
 |-- description: string (nullable = true)
 |-- location_description: string (nullable = true)
 |-- arrest: string (nullable = true)
 |-- domestic: string (nullable = true)
 |-- beat: string (nullable = true)
 |-- district: string (nullable = true)
 |-- ward: string (nullable = true)
 |-- fbi_code: string (nullable = true)
 |-- x_coordinate: string (nullable = true)
 |-- y_coordinate: string (nullable = true)
 |-- updated_on: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- location: string (nulla

In [12]:
#df_ml.select([count(when(isnan(c), c)).alias(c) for c in df_ml.columns]).toPandas()

In [13]:
categoricalColumns = ['domestic']
numericCols = ['year', 'month', 'day', 'hour','minute', 'latitude',
               'longitude', 'isStreet', 'isAV',
               'isBLVD', 'isRD', 'isPL', 'isBROADWAY',
               'isPKWY', 'duree_day', 'minute','dayofmonth',
               'dayofyear','dayofweek','Temperature',
               'pct_housing_crowded',
               'pct_households_below_poverty',
               'pct_age16_unemployed',
               'pct_age25_no_highschool',
               'pct_not_working_age',
               'per_capita_income',
               'hardship_index',
                'Chicago_broken clouds',
                'Chicago_drizzle',
                 'Chicago_few clouds',
                'Chicago_fog',
                 'Chicago_haze',
                 'Chicago_heavy intensity drizzle',
                 'Chicago_heavy intensity rain',
                 'Chicago_heavy snow',
                 'Chicago_light intensity drizzle',
                 'Chicago_light rain',
                 'Chicago_light rain and snow',
                 'Chicago_light snow',
                 'Chicago_mist',
                 'Chicago_moderate rain',
                 'Chicago_overcast clouds',
                 'Chicago_proximity thunderstorm',
                 'Chicago_scattered clouds',
                 'Chicago_sky is clear',
                 'Chicago_snow',
                 'Chicago_thunderstorm',
                 'Chicago_thunderstorm with heavy rain',
                 'Chicago_thunderstorm with light rain',
                 'Chicago_thunderstorm with rain',
                'Chicago_very heavy rain'
]


In [14]:
rf = RandomForestClassifier(labelCol='label', featuresCol='features',numTrees=150, maxDepth=8)

In [15]:
stages = []
label_stringIdx = StringIndexer(inputCol = 'primary_type', outputCol = 'label')
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages.append(stringIndexer)
    stages.append(encoder)
stages.append(label_stringIdx)
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages.append(assembler)
stages.append(rf)

In [16]:
df_train, df_test = df_ml.randomSplit([0.7, 0.3])

In [17]:
pipeline_and_model = Pipeline(stages = stages)

In [18]:
%%time
pipeline_model = pipeline_and_model.fit(df_train)

CPU times: user 126 ms, sys: 13.3 ms, total: 139 ms
Wall time: 25min 41s


In [19]:
%%time
df_test_pred = pipeline_model.transform(df_test)

CPU times: user 11.1 ms, sys: 4.64 ms, total: 15.7 ms
Wall time: 376 ms


In [20]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [21]:
predictions = df_test_pred.select(col("label"), col("prediction"))

In [22]:
%%time
predictions_ = predictions.cache()

CPU times: user 838 µs, sys: 89 µs, total: 927 µs
Wall time: 196 ms


In [23]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

In [24]:
accuracy = evaluator.evaluate(predictions_)

In [25]:
print(accuracy)

0.6671939821466044
